In [1]:
import pandas as pd
import numpy as np
import cv2
import dlib
import os

In [2]:
face_cascade = cv2.CascadeClassifier('../face_recognition/haarcascade_frontalface_default.xml')
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [5]:
IMG_PATH="../facial_expressions/"

In [184]:
emotions=["anger", "disgust", "fear", "happy", "sad", "surprise", "neutral"]

In [111]:
df=pd.read_csv(IMG_PATH+"icml_face_data.csv")

In [113]:
df=df.replace(to_replace="PrivateTest",value="Testing")
df=df.replace(to_replace="PublicTest",value="Training")

In [114]:
df

,emotion,Usage,pixels
0,0,Training,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
1,0,Training,151 150 147 155 148 133 111 140 170 174 182 15...
2,2,Training,231 212 156 164 174 138 161 173 182 200 106 38...
3,4,Training,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
4,6,Training,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...
...,...,...,...
35882,6,Testing,50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...
35883,3,Testing,178 174 172 173 181 188 191 194 196 199 200 20...
35884,0,Testing,17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...
35885,3,Testing,30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...


In [115]:
data={"Training":[],"Testing":[]}

In [116]:
for i in range(len(df)):
    print("\r",i+1,end=" ")
    row=df.iloc[i]
    img=np.array(list(map(np.uint8,row[2].split()))).reshape(48,48)
    data[row[1]].append([row[0],ret_keypoints(img)])

 35887  4965      32797  

In [138]:
import pickle

In [139]:
with open("data.pkl","wb") as f:
    pickle.dump(data, f)

In [ ]:
# with open('data.pkl', 'rb') as f:
#     data = pickle.load(f)

In [140]:
face_dim=64

In [141]:
import matplotlib.pyplot as plt

In [142]:
def ret_keypoints(image):
    gray=image
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, (face_dim, face_dim))
#     faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    param=[0,0,face_dim,face_dim]
    shape=predictor(gray,dlib.rectangle(*param))
    xlist=[]
    ylist=[]
    for i in range(68):
        xlist.append(np.float32(shape.part(i).x))
        ylist.append(np.float32(shape.part(i).y))
    xmean = np.mean(xlist)
    ymean = np.mean(ylist)
#     plt.imshow(gray,cmap='gray')
#     plt.scatter(xlist,ylist, marker='.')
#     plt.show()
    xcentral = [(x-xmean) for x in xlist]
    ycentral = [(y-ymean) for y in ylist]
    res=[]
    for (x,y) in zip(xcentral,ycentral):
        res.append(x)
        res.append(y)
    return (np.asarray(res)/face_dim+1)/2

In [143]:
# s=np.arange(len(XXD))
# for i in range(5):
#     np.random.shuffle(s)
#     XXD=XXD[s]
#     YYD=YYD[s]

In [144]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

In [211]:
model=Sequential()
model.add(Dense(1024,activation='relu', input_shape=(68*2,)))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(256,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Dense(len(emotions),activation='softmax'))

In [212]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 1024)              140288    
_________________________________________________________________
batch_normalization_8 (Batch (None, 1024)              4096      
_________________________________________________________________
dropout_8 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_11 (Dense)             (None, 512)               524800    
_________________________________________________________________
batch_normalization_9 (Batch (None, 512)               2048      
_________________________________________________________________
dropout_9 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 256)              

In [213]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [188]:
XDB=np.vstack(np.asarray(data["Training"])[:,1])
YDY=np.asarray(data["Training"])[:,0]
YDB=np.zeros((len(YDY),len(emotions)))
for i in range(len(YDB)):
    YDB[i][YDY[i]]=1
XTB=np.vstack(np.asarray(data["Testing"])[:,1])
YTY=np.asarray(data["Testing"])[:,0]
YTB=np.zeros((len(YTY),len(emotions)))
for i in range(len(YTB)):
    YTB[i][YTY[i]]=1

In [214]:
model.fit(XDB,YDB,batch_size=64,epochs=100,validation_data=(XTB,YTB))

Train on 32298 samples, validate on 3589 samples
Epoch 1/100
32298/32298 [==============================] - 8s 250us/sample - loss: 1.7771 - accuracy: 0.3559 - val_loss: 1.5882 - val_accuracy: 0.3834
Epoch 2/100
32298/32298 [==============================] - 7s 219us/sample - loss: 1.5203 - accuracy: 0.4204 - val_loss: 1.5737 - val_accuracy: 0.4009
Epoch 3/100
32298/32298 [==============================] - 7s 232us/sample - loss: 1.4671 - accuracy: 0.4366 - val_loss: 1.4173 - val_accuracy: 0.4620
Epoch 4/100
32298/32298 [==============================] - 7s 214us/sample - loss: 1.4435 - accuracy: 0.4443 - val_loss: 1.7628 - val_accuracy: 0.2672
Epoch 5/100
32298/32298 [==============================] - 7s 223us/sample - loss: 1.4316 - accuracy: 0.4524 - val_loss: 1.6560 - val_accuracy: 0.3527
Epoch 6/100
32298/32298 [==============================] - 7s 210us/sample - loss: 1.4184 - accuracy: 0.4596 - val_loss: 1.7413 - val_accuracy: 0.3062
Epoch 7/100
32298/32298 [====================

32298/32298 [==============================] - 6s 177us/sample - loss: 1.2886 - accuracy: 0.4997 - val_loss: 1.3976 - val_accuracy: 0.4634
Epoch 55/100
32298/32298 [==============================] - 6s 178us/sample - loss: 1.2894 - accuracy: 0.5021 - val_loss: 1.5204 - val_accuracy: 0.4026
Epoch 56/100
32298/32298 [==============================] - 6s 188us/sample - loss: 1.2881 - accuracy: 0.5018 - val_loss: 1.7603 - val_accuracy: 0.3647
Epoch 57/100
32298/32298 [==============================] - 6s 192us/sample - loss: 1.2851 - accuracy: 0.5023 - val_loss: 1.3823 - val_accuracy: 0.4458
Epoch 58/100
32298/32298 [==============================] - 6s 190us/sample - loss: 1.2860 - accuracy: 0.4998 - val_loss: 1.4314 - val_accuracy: 0.4670
Epoch 59/100
32298/32298 [==============================] - 7s 217us/sample - loss: 1.2872 - accuracy: 0.4999 - val_loss: 1.3224 - val_accuracy: 0.4845
Epoch 60/100
32298/32298 [==============================] - 9s 277us/sample - loss: 1.2794 - accuracy

In [215]:
(model.predict(XTB).argmax(axis=1)==YTB.argmax(axis=1)).sum()/len(XTB)

0.4608526051825021

In [216]:
model.predict(XTB).argmax(axis=1)

array([4, 4, 4, ..., 4, 4, 4])

In [217]:
YTB.argmax(axis=1)

array([0, 5, 6, ..., 0, 3, 2])

In [224]:
from udp_streamer import *

In [249]:
handler = udp_handler()
handler.make_listener('0.0.0.0',5555)

In [261]:
def get_img():
    global handler,img
    buffer = handler.get_data()
    if buffer is not None:
        try:
            npimg = np.frombuffer(buffer, dtype=np.uint8)
            img = cv2.imdecode(npimg, 1)
        except Exception as e:
            print(e)
    return img

In [264]:
# cam = cv2.VideoCapture(0)

while True:
#     ret, img = cam.read()
    img=get_img()
    gimg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gimg, 1.3, 5)
    for (x,y,w,h) in faces:
        landmarks=[]
        svx,svy=x,y
        gray=gimg[y:y+h,x:x+w]
        try:
            gray=cv2.resize(gray, (face_dim, face_dim))
        except:
            print(gray,x,y,w,h)
            continue
        shape=predictor(gray,dlib.rectangle(0,0,face_dim,face_dim))
        xlist=[]
        ylist=[]
        for i in range(68):
            xp=shape.part(i).x
            yp=shape.part(i).y
            cv2.circle(gray, (xp, yp), 2, (255, 255, 255), -1)
            xlist.append(float(xp))
            ylist.append(float(yp))
        cv2.imshow('gray',gray)
        xmean = np.mean(xlist)
        ymean = np.mean(ylist)
        xcentral = [(x-xmean) for x in xlist]
        ycentral = [(y-ymean) for y in ylist]
        res=[]
        for (x,y) in zip(xcentral,ycentral):
            res.append(x)
            res.append(y)
        landmarks.append((np.asarray(res)/face_dim+1)/2)
        if len(landmarks)>0:
            y_out=model.predict(np.asarray(landmarks))
            res=np.argmax(y_out,axis=1)
            for r in res:
                cv2.putText(img,emotions[r],(svx,svy),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,0),2,cv2.LINE_AA)
    cv2.imshow('webcam', img)
    if cv2.waitKey(1) & 0xff == 27:
        break

cam.release()
cv2.destroyAllWindows()

In [21]:
cam.release()
cv2.destroyAllWindows()